# ESP Experiment

This notebook defines an agent to play the ESP card game. It runs on Python 3.8.

### Contents:
1. Imports
1. Interfaces, utilities and experiment constants
    1. Agent interface
    2. Experiment constants
    3. Utilities for running the experiments
2. Specific Agents
    1. An agent that learns from observation
    2. Greedy action
    3. Optimal action
    4. TS action
    5. IDS action
    6. Lagrangian-IDS action
3. Sweeps
    1. Information Agent vs $\gamma$
    2. Greedy, Optimal, Lagrangian-IDS action EV vs Initial Deck

In [3]:
#@title Imports
from typing import Optional
from __future__ import absolute_import, division, print_function
import os, sys, pdb, pickle
from multiprocessing import Pool
from fractions import Fraction
import time, math
from copy import deepcopy

import abc
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import pylab
import scipy

colors = {0: 'b',
          1: 'g',
          2: 'r',
          3: 'c',
          4: 'm',
          5: 'y',
          6: 'k'}

# Interfaces, utilities and experiment constants
Use this as the base class for agents.

In [4]:
#@title Agent interface
class Agent(metaclass=abc.ABCMeta):

  @abc.abstractmethod
  def reset(self, initial: tuple, rng: np.random.Generator) -> None:
    # Reset the agent.
    raise NotImplementedError
 
  @abc.abstractmethod
  def update(self, action: int, obs: np.ndarray) -> None:
    # Update agent state.
    raise NotImplementedError

  @abc.abstractmethod
  def select_action(self) -> int:
    # Select an action.
    raise NotImplementedError

In [45]:
#@title Experiment constants
TYPES = 3
NUM_CARDS = 3
initial = (3, 3, 3, 3)
seed = 0
random_generator = np.random.default_rng(seed)

### Run randomly simulated experiments

In [46]:
def run_ESP_experiment(
    num_sims: int, 
    agent,
    types_of_cards: int = TYPES,
    num_of_each_type: int = NUM_CARDS,
    plot_results=False):
  '''
  Evaluate agents based on random simulations
  '''
  num_action = types_of_cards
  sims = range(num_sims)
  num_timesteps = types_of_cards * num_of_each_type
  timesteps = range(num_timesteps)
  actions = list(range(num_action))
  rewards = []

  action_count = []

  for sim in sims:
    rng = np.random.default_rng(sim)
    agent.reset(rng)
    true = [i for i in range(TYPES) for j in range(NUM_CARDS)]
    rng.shuffle(true)
    # TODO: Change regret calculation method
    action_count += [[num_timesteps*[0] for a in actions]]
    rewards += [num_timesteps * [0]]
    for timestep in timesteps:
      a = agent.select_action()
      obs = (true[timestep] == a)
      agent.update(a, obs)

      action_count[sim][a][timestep] = 1
      rewards[sim][timestep] += obs

    action_count1 = [[float(action_count[0][a][t]) 
    for t in range(num_timesteps)] for a in actions]
    rewards1 = sum(rewards[0])
  action_count = [[sum([float(action_count[sim][a][t]) 
    for sim in sims]) / num_sims 
    for t in range(num_timesteps)] for a in actions]
  expected_reward = sum(sum(rewards,[])) / num_sims

  if plot_results:
    # plot action frequencies averaged over simulations
    for a in actions:
      plt.plot(timesteps,
              pd.Series(action_count[a]).rolling(10, min_periods=1).mean(),
              colors[a], label='$a =$' + str(a+1))
    plt.axis([0,num_timesteps+1,0.0,1.01])
    plt.xlabel(r'time $t$', fontsize=20)
    plt.ylabel('$\mathbb{P}(A_t = a|\mathcal{E})$', fontsize=20)
    pylab.legend(loc='best')
    plt.show()

    # plot action counts over single simulation
    for a in actions:
      plt.plot(timesteps, np.cumsum(action_count1[a]), colors[a], label='$a =$' + str(a+1))
    plt.axis([0,num_timesteps+1,0.0,num_timesteps+1])
    plt.xlabel(r'time $t$', fontsize=20)
    plt.ylabel('$N_{t,a}$', fontsize=20)
    pylab.legend(loc='best')
    plt.show()

    # plot action counts averaged over simulations
    for a in actions:
      plt.plot(timesteps, np.cumsum(action_count[a]), colors[a], label='$a =$' + str(a+1))
    plt.axis([0,num_timesteps+1,0.0,num_timesteps+1])
    plt.xlabel(r'time $t$', fontsize=20)
    plt.ylabel('$\mathrm{E}[N_{t,a}|\mathcal{E}]$', fontsize=20)
    pylab.legend(loc='best')
    plt.show()

  return action_count, expected_reward

# Agents

In [47]:
#@title An agent that learns from observation
class LearnFromObsAgent(Agent):
  def __init__(self, initial=None, compute_action=None, rng=None):
    if initial: self.reset(initial)
    self._compute_action = compute_action
    self._rng = rng

  def reset(self, initial, rng=None):
    self._initial = initial
    self._num_type  = len(initial) # number of card types
    self._total = sum(self._initial) # total number of cards
    self._states = {self._initial: Fraction(1)}
    self._correct = 0
    self._type_prob = [ Fraction(x, sum(self._initial)) for x in self._initial]
    self._rng = rng if rng else np.random.default_rng()

  def update_type_prob(self):
    if self._total:
      self._type_prob = [0] * self._num_type
      for comp, prob in self._states.items():
        for i in range(len(comp)):
          self._type_prob[i] += prob * comp[i] / self._total
      if sum(self._type_prob) != 1:
        print('Warning: sum of group probabilities %f != 1.0'%sum(self.group_prob))
    
  def update(self, action: int, correct: bool):
    # update state
    self._correct += correct
    self._total -= 1
    next_states = {}
    for comp, prob in self._states.items():
      if correct and comp[action]:
        posterior = Fraction(comp[action], sum(comp)) * prob / self._type_prob[action]
        cl = list(comp)
        cl[action] -= 1
        cl = tuple(cl)
        if cl not in next_states: next_states[cl] = 0
        next_states[cl] += posterior
      if not correct and sum(comp) - comp[action]:
        posterior_0 = (1 - Fraction(comp[action], sum(comp))) * prob / (1 - self._type_prob[action])
        for other in range(self._num_type):
          if other == action or not comp[other]: continue
          posterior = posterior_0 * Fraction(comp[other], sum(comp) - comp[action])
          cl = list(comp)
          cl[other] -= 1
          cl = tuple(cl)
          if cl not in next_states: next_states[cl] = Fraction(0)
          next_states[cl] += posterior 
    self._states = next_states
    if not len(self._states): return False
    if sum(self._states.values()) != 1:
      print('Warning: sum of state probabilities %f != 1.0'%sum(self.states.values()))
    self.update_type_prob()

  def select_action(self):
    return self._compute_action(self._rng, self._type_prob, len(self._states))
  
  def set_composition(self, composition):
    # 
    example_element = list(composition.keys())[0]
    self._num_type = len(example_element)
    self._total = sum(example_element)
    self._states = composition
    self._correct = 0
    self.update_type_prob()

  def serialize(self):
    #
    return 'x'.join(map(str, self._initial)) + ' ' + ' '.join(sorted(['|'.join(map(str, k)) + ':%s'%str(v) for k,v in self._states.items()]))

  def get_state_entropy(self, total=False):
    entropy = 0.0
    for k,v in self._states.items():
        micro_per_macro = math.factorial(sum(k)) / np.prod(map(math.factorial, k)) if total else 1.0
        entropy -= v * np.log2(v / micro_per_macro)
    return entropy
  
  def ev(self, memo=None):
    # Evaluate agents via recursion, supposedly faster than deck_ev. 
    # Not used for now.
    if self._total <= 0: return 0
    if memo is None: memo = {}
    serial = self.serialize()
    if serial in memo: return memo[serial]
    action = self.select_action()
    
    copy0 = deepcopy(self)
    if copy0.update(action, False): ev0 = copy0.ev(memo)
    else: ev0 = 0
    copy1 = deepcopy(self)
    if copy1.update(action,  True): ev1 = copy1.ev(memo) + 1
    else: ev1 = 0
    
    ev = (1 - self._type_prob[action]) * ev0 + self._type_prob[action] * ev1
    memo[serial] = ev
    print(ev)
    return ev

  def run(self, deck):
    #
    self.reset(tuple(np.bincount(deck)))
    for card in deck:
        action = self.select_action()
        self.update(action, action==card)
    return self._correct

In [48]:
#@title Utilities for running the experiments
def random_argmax(rng, scores: np.ndarray):
  probs = (scores==scores.max()).astype(np.float32)
  probs /= probs.sum()
  return rng.choice(np.arange(scores.size), p=probs)

def generate_all_decks(initial):
  '''
  Given a compressed representation of the composition of cards, 
  generate a list of all decks
  '''
  all_decks = []
  if sum(initial) == 1:
    for i in range(len(initial)):
      if initial[i] > 0: return [[i]]
  for i in range(len(initial)):
    if initial[i] > 0:
      initial[i] -= 1
      recur_decks = generate_all_decks(initial)
      initial[i] += 1
      all_decks += [[i] + deck for deck in recur_decks]
  return all_decks

all_decks = generate_all_decks([3,3,3])
print('Number of Unique Decks:', len(all_decks))

from functools import partial
def f(l, a):
  #
  return a.run(l)

def chunks(l, n):
  #
  for i in range(0, len(l), n):
    yield l[i:i+n]

def deck_ev(initial, compute_action, verbose=True, **kwargs):
  #
  nproc = 8
  p = Pool(nproc)
  batch_size = 2 * nproc
  a = LearnFromObsAgent(initial, compute_action, **kwargs)

  all_decks = generate_all_decks(list(initial))
  correct_guesses = []
  for idx, decks in enumerate(chunks(all_decks, batch_size)):
      correct_guesses += p.map(partial(f, a=a), decks)
      if verbose:
        print('\rOn deck %s (%04d/%04d) | EV %.6f'%(''.join(map(str, decks[-1])), min((idx+1)*batch_size, len(all_decks)), len(all_decks), np.mean(correct_guesses)), end='')
        sys.stdout.flush()
  if verbose:
      print()
  p.terminate()
  return np.mean(correct_guesses)

Number of Unique Decks: 1680


In [49]:
#@title Greedy Action
def greedy_action(rng, type_prob, num_states):
  return np.argmax(type_prob)
  # return random_argmax(rng, np.array(type_prob)) # maybe due to the persistent conjecture, this is often worse than the above, but not always

# Test Script
a = LearnFromObsAgent(compute_action=greedy_action)
a.reset((3,3,3))
action1 = a.select_action()
a.update(action1, True)
action2 = a.select_action()
a.update(action2, False)
print('First guess: %d | Second guess: %d | States:'%(action1, action2), a._states)

First guess: 0 | Second guess: 1 | States: {(1, 3, 3): Fraction(2, 5), (2, 3, 2): Fraction(3, 5)}


In [50]:
t0 = time.time()
ev = deck_ev(initial, greedy_action, random_generator)
print(initial, '=> Expected Correct Guesses: %.15f (%s)'%(ev, str(ev)))
print('Took %.3f seconds'%(time.time() - t0))

On deck 333222111000 (369600/369600) | EV 4.249045
(3, 3, 3, 3) => Expected Correct Guesses: 4.249044913419914 (4.2490449134199135)
Took 756.660 seconds


In [11]:
#@title TS Action
def ts_action(rng, type_prob, num_states):
  num_type = len(type_prob)
  return rng.choice(range(num_type), p=type_prob)

# Test Script
a = LearnFromObsAgent(compute_action=ts_action)
a.reset((3,3,3))
action1 = a.select_action()
a.update(action1, True)
action2 = a.select_action()
a.update(action2, False)
print('First guess: %d | Second guess: %d | States:'%(action1, action2), a._states)

First guess: 1 | Second guess: 2 | States: {(2, 2, 3): Fraction(3, 5), (3, 1, 3): Fraction(2, 5)}


In [12]:
t0 = time.time()
ev = deck_ev(initial, ts_action, random_generator)
print(initial, '=> Expected Correct Guesses: %.15f (%s)'%(ev, str(ev)))
print('Took %.3f seconds'%(time.time() - t0))

/tmp/ipykernel_118236/3555268770.py:4: DeprecationWarning: Fraction.__float__ returned non-float (type numpy.float64).  The ability to return an instance of a strict subclass of float is deprecated, and may be removed in a future version of Python.
  return rng.choice(range(num_type), p=type_prob)
/tmp/ipykernel_118236/3555268770.py:4: DeprecationWarning: Fraction.__float__ returned non-float (type numpy.float64).  The ability to return an instance of a strict subclass of float is deprecated, and may be removed in a future version of Python.
  return rng.choice(range(num_type), p=type_prob)
/tmp/ipykernel_118236/3555268770.py:4: DeprecationWarning: Fraction.__float__ returned non-float (type numpy.float64).  The ability to return an instance of a strict subclass of float is deprecated, and may be removed in a future version of Python.
  return rng.choice(range(num_type), p=type_prob)
/tmp/ipykernel_118236/3555268770.py:4: DeprecationWarning: Fraction.__float__ returned non-float (type 

On deck 33221100 (2520/2520) | EV 2.234921
(2, 2, 2, 2) => Expected Correct Guesses: 2.234920634920635 (2.234920634920635)
Took 2.264 seconds


In [51]:
#@title IDS Action
def ids_action(rng, type_prob, num_states=0):
  num_type = len(type_prob)
  mutual_infos = [scipy.stats.entropy([1-float(p), float(p)]) for p in type_prob]
  mutual_infos = np.array(mutual_infos)
  shortfall = max(type_prob) - np.array(type_prob)
  minimum = float('inf')
  min_actions = [-1, -1]
  min_alpha = -1
  for a in range(num_type):
    func = np.float64((shortfall[a])**2) / (mutual_infos[a])
    # print(mutual_infos)
    # func = 1 / (mutual_infos[a])
    if func < minimum:
      minimum, min_actions, min_alpha = func, [a, 0], 1
    for b in range(a, num_type):
      if mutual_infos[a] == 0 and mutual_infos[b] == 0: continue
      func = lambda x: np.float64((x * shortfall[a] + (1 - x) * shortfall[b])**2) / (x * mutual_infos[a] + (1 - x) * mutual_infos[b])
      res = scipy.optimize.minimize(func, 0, bounds=[(0, 1)])
      if res.fun < minimum:
        minimum, min_actions, min_alpha = res.fun, [a, b], res.x[0]
  if min_alpha == -1:
    return np.argmax(type_prob)
  if min_alpha != 0 and min_alpha != 1: print(min_alpha)
  return rng.choice(min_actions, p=[min_alpha, 1 - min_alpha])

# Test Script
a = LearnFromObsAgent(compute_action=ids_action)
a.reset((3,3,3))
action1 = a.select_action()
a.update(action1, True)
action2 = a.select_action()
a.update(action2, False)
print('First guess: %d | Second guess: %d | States:'%(action1, action2), a._states)


First guess: 0 | Second guess: 1 | States: {(1, 3, 3): Fraction(2, 5), (2, 3, 2): Fraction(3, 5)}


In [52]:
t0 = time.time()
ev = deck_ev(initial, ids_action, random_generator)
print(initial, '=> Expected Correct Guesses: %.15f (%s)'%(ev, str(ev)))
print('Took %.3f seconds'%(time.time() - t0))

/tmp/ipykernel_118236/348619590.py:4: DeprecationWarning: Fraction.__float__ returned non-float (type numpy.float64).  The ability to return an instance of a strict subclass of float is deprecated, and may be removed in a future version of Python.
  mutual_infos = [scipy.stats.entropy([1-float(p), float(p)]) for p in type_prob]
/tmp/ipykernel_118236/348619590.py:4: DeprecationWarning: Fraction.__float__ returned non-float (type numpy.float64).  The ability to return an instance of a strict subclass of float is deprecated, and may be removed in a future version of Python.
  mutual_infos = [scipy.stats.entropy([1-float(p), float(p)]) for p in type_prob]
/tmp/ipykernel_118236/348619590.py:4: DeprecationWarning: Fraction.__float__ returned non-float (type numpy.float64).  The ability to return an instance of a strict subclass of float is deprecated, and may be removed in a future version of Python.
  mutual_infos = [scipy.stats.entropy([1-float(p), float(p)]) for p in type_prob]
/tmp/ipyke

On deck 000112233231 (0048/369600) | EV 4.687500

/tmp/ipykernel_118236/348619590.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  func = np.float64((shortfall[a])**2) / (mutual_infos[a])
/tmp/ipykernel_118236/348619590.py:11: RuntimeWarning: invalid value encountered in double_scalars
  func = np.float64((shortfall[a])**2) / (mutual_infos[a])


On deck 000112323132 (0064/369600) | EV 4.906250

/tmp/ipykernel_118236/348619590.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  func = np.float64((shortfall[a])**2) / (mutual_infos[a])
/tmp/ipykernel_118236/348619590.py:11: RuntimeWarning: invalid value encountered in double_scalars
  func = np.float64((shortfall[a])**2) / (mutual_infos[a])
/tmp/ipykernel_118236/348619590.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  func = np.float64((shortfall[a])**2) / (mutual_infos[a])
/tmp/ipykernel_118236/348619590.py:11: RuntimeWarning: invalid value encountered in double_scalars
  func = np.float64((shortfall[a])**2) / (mutual_infos[a])
/tmp/ipykernel_118236/348619590.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  func = np.float64((shortfall[a])**2) / (mutual_infos[a])
/tmp/ipykernel_118236/348619590.py:11: RuntimeWarning: invalid value encountered in double_scalars
  func = np.float64((shortfall[a])**2) / (mutual_infos[a])


...

On deck 000123221313 (0304/369600) | EV 5.207237

/tmp/ipykernel_118236/348619590.py:18: RuntimeWarning: divide by zero encountered in divide
  func = lambda x: np.float64((x * shortfall[a] + (1 - x) * shortfall[b])**2) / (x * mutual_infos[a] + (1 - x) * mutual_infos[b])
/tmp/ipykernel_118236/348619590.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  func = np.float64((shortfall[a])**2) / (mutual_infos[a])
/tmp/ipykernel_118236/348619590.py:18: RuntimeWarning: divide by zero encountered in divide
  func = lambda x: np.float64((x * shortfall[a] + (1 - x) * shortfall[b])**2) / (x * mutual_infos[a] + (1 - x) * mutual_infos[b])
/tmp/ipykernel_118236/348619590.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  func = np.float64((shortfall[a])**2) / (mutual_infos[a])
/tmp/ipykernel_118236/348619590.py:11: RuntimeWarning: invalid value encountered in double_scalars
  func = np.float64((shortfall[a])**2) / (mutual_infos[a])
/tmp/ipykernel_118236/348619590.py:11: RuntimeWarning: invalid value encountered

On deck 000123321231 (0336/369600) | EV 5.247024

/tmp/ipykernel_118236/348619590.py:11: RuntimeWarning: invalid value encountered in double_scalars
  func = np.float64((shortfall[a])**2) / (mutual_infos[a])
/tmp/ipykernel_118236/348619590.py:18: RuntimeWarning: divide by zero encountered in divide
  func = lambda x: np.float64((x * shortfall[a] + (1 - x) * shortfall[b])**2) / (x * mutual_infos[a] + (1 - x) * mutual_infos[b])


On deck 333222111000 (369600/369600) | EV 4.249045
(3, 3, 3, 3) => Expected Correct Guesses: 4.249044913419914 (4.2490449134199135)
Took 3884.765 seconds


In [53]:
class OptimalAgent(object):
    def __init__(self, initial=None):
        if initial: self.reset(initial)
        
    def reset(self, initial):
        self.initial = initial
        self.group_num = len(initial)
        self.size = sum(self.initial)
        self.states = {self.initial:Fraction(1)}
        self.correct = 0
        self.group_prob = [ Fraction(x, sum(self.initial)) for x in self.initial ]

    def set_composition(self, composition):
        example_element = list(composition.keys())[0]
        self.group_num = len(example_element)
        self.size = sum(example_element)
        self.states = composition
        self.correct = 0
        self.update_group_prob()

    def serialize(self):
        return 'x'.join(map(str, self.initial)) + ' ' + ' '.join(sorted(['|'.join(map(str, k)) + ':%s'%str(v) for k,v in self.states.items()]))
        
    def expected_correct(self, guess, memo, verbose=False):
        if self.size <= 0: return 0
        
        copy0 = OptimalAgent(initial=self.initial)
        copy0.set_composition(deepcopy(self.states))
        if copy0.update_state(guess, False): ev0 = copy0.make_guess_aux(memo)[1]
        else: ev0 = 0
        
        copy1 = OptimalAgent(initial=self.initial)
        copy1.set_composition(deepcopy(self.states))
        if copy1.update_state(guess,  True): ev1 = copy1.make_guess_aux(memo)[1] + 1
        else: ev1 = 0
            
        ev = (1 - self.group_prob[guess]) * ev0 + self.group_prob[guess] * ev1
            
        if verbose and self.size == 3:
            print(self.states)
            print('Guess:', guess, ' - ev0:', ev0, ' - ev1:', ev1, ' - ev:', ev)
            
        return ev
    
    def make_guess_aux(self, memo, verbose=False):
        serial = self.serialize()
        if serial in memo: return memo[serial]
        if sum(np.array(self.group_prob) > 0) <= 2:
            pick = np.argmax(self.group_prob)
            ret = (pick, self.expected_correct(pick, memo))
            memo[serial] = ret
            return ret
        best_ev = 0
        best_pick = 0
        for pick in range(self.group_num):
            ev = self.expected_correct(pick, memo, verbose=verbose)
            if ev > best_ev:
                best_ev = ev
                best_pick = pick
        ret = (best_pick, best_ev)
        memo[serial] = ret
        return ret
    
    def ev(self):
        memo = {}
        return self.make_guess_aux(memo)[1]

    def update_group_prob(self):
        if self.size:
            self.group_prob = [0] * self.group_num
            for k,v in self.states.items():
                for i in range(len(k)):
                    self.group_prob[i] += v * k[i] / self.size
            if sum(self.group_prob) != 1:
                print('Warning: sum of group probabilities %f != 1.0'%sum(self.group_prob))

    def update_state(self, guess, correct):
        self.correct += correct
        self.size -= 1
        next_states = {}
        for k,v in self.states.items():
            if correct and k[guess]:
                bayes = Fraction(k[guess], sum(k)) * v / self.group_prob[guess]
                kl = list(k)
                kl[guess] -= 1
                kl = tuple(kl)
                if kl not in next_states: next_states[kl] = Fraction(0)
                next_states[kl] += bayes
            if not correct and sum(k) - k[guess]:
                bayes0 = (1 - Fraction(k[guess], sum(k))) * v / (1 - self.group_prob[guess])
                for other in range(self.group_num):
                    if other == guess or not k[other]: continue
                    bayes = bayes0 * Fraction(k[other], sum(k) - k[guess])
                    kl = list(k)
                    kl[other] -= 1
                    kl = tuple(kl)
                    if kl not in next_states: next_states[kl] = Fraction(0)
                    next_states[kl] += bayes
        self.states = next_states
        if not len(self.states): return False
        if sum(self.states.values()) != 1:
            print('Warning: sum of state probabilities %f != 1.0'%sum(self.states.values()))
        self.update_group_prob()
        return True
    
    def make_guess(self, verbose=False):
        memo = {}
        return self.make_guess_aux(memo, verbose=verbose)[0]

# Test Script
a = OptimalAgent()
a.reset((1,1,1))
guess1 = a.make_guess()
a.update_state(guess1, True)
guess2 = a.make_guess()
a.update_state(guess2, False)
print('First guess: %d | Second guess: %d | States:'%(guess1, guess2), a.states)

First guess: 0 | Second guess: 1 | States: {(0, 1, 0): Fraction(1, 1)}


In [54]:
t0 = time.time()
a = OptimalAgent()
a.reset(initial)
ev = a.ev()
print(initial, '=> EV: %.15f (%s)'%(ev, str(ev)))
print('Took %.3f seconds'%(time.time() - t0))

(3, 3, 3, 3) => EV: 4.280386904761905 (143821/33600)
Took 84.085 seconds


## Evaluation via Random Simulation

In [17]:
#@title Test evaluation
num_sims = 1000 # number of simulations over which to average

In [ ]:
greedy_agent = LearnFromObsAgent(greedy_action, initial)

action_count_greedy, reward_greedy = run_ESP_experiment(
    num_sims,
    greedy_agent)

reward_greedy

4.221

In [ ]:
ts_agent = LearnFromObsAgent(ts_action, initial)

action_count_ts, reward_ts = run_ESP_experiment(
    num_sims,
    ts_agent)
reward_ts

3.426

In [ ]:
ids_agent = LearnFromObsAgent(ids_action, initial)

action_count_ids, reward_ids = run_ESP_experiment(
    num_sims,
    ids_agent)
reward_ids

<ipython-input-76-a8043ab9b687>:10: RuntimeWarning: invalid value encountered in double_scalars
<ipython-input-76-a8043ab9b687>:14: RuntimeWarning: invalid value encountered in true_divide
<ipython-input-76-a8043ab9b687>:14: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.8/dist-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
<ipython-input-76-a8043ab9b687>:10: RuntimeWarning: divide by zero encountered in double_scalars
<ipython-input-76-a8043ab9b687>:14: RuntimeWarning: invalid value encountered in multiply


4.273